In [ ]:
#9ezXtdh1pB2PhMu5oWrd1Q client id
#client_secret = kZV7lztFgF9aUtNPzN0rS9lXReQ0xg
# u/Early_Impression5129

In [1]:
!pip install praw

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 3.3 MB/s eta 0:00:00a 0:00:01


In [ ]:
import praw
import pandas as pd
from datetime import datetime
import time

# Reddit API credentials (replace with your own)
reddit = praw.Reddit(
    client_id='9ezXtdh1pB2PhMu5oWrd1Q',
    client_secret='kZV7lztFgF9aUtNPzN0rS9lXReQ0xg',
    user_agent='u/Early_Impression5129'
)

def scrape_subreddit(subreddit_name, post_limit=1000, comment_limit=10):
    subreddit = reddit.subreddit(subreddit_name)
    posts_data = []

    for post in subreddit.new(limit=post_limit):
        # Extract post data
        post_data = {
            'subreddit': subreddit_name,
            'post_id': post.id,
            'title': post.title,
            'body': post.selftext,
            'score': post.score,
            'num_comments': post.num_comments,
            'created_utc': datetime.fromtimestamp(post.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
            'author': str(post.author),
            'url': post.url
        }
        
        # Extract comments
        post.comments.replace_more(limit=0)  # Flatten comment tree
        comments = []
        for comment in post.comments.list()[:comment_limit]:
            comments.append({
                'comment_id': comment.id,
                'comment_body': comment.body,
                'comment_score': comment.score,
                'comment_author': str(comment.author),
                'comment_created_utc': datetime.fromtimestamp(comment.created_utc).strftime('%Y-%m-%d %H:%M:%S')
            })
        
        post_data['comments'] = comments
        posts_data.append(post_data)
        
        # Sleep to respect Reddit's rate limits
        time.sleep(2)

    return posts_data

def main():
    subreddits = ['stocks', 'investing']
    all_data = []

    for subreddit in subreddits:
        print(f"Scraping r/{subreddit}...")
        subreddit_data = scrape_subreddit(subreddit)
        all_data.extend(subreddit_data)
        print(f"Finished scraping r/{subreddit}. Total posts scraped: {len(subreddit_data)}")

    # Convert to DataFrame
    df = pd.json_normalize(all_data, 'comments', ['subreddit', 'post_id', 'title', 'body', 'score', 'num_comments', 'created_utc', 'author', 'url'],
                           record_prefix='comment_')

    # Save to CSV
    filename = f"reddit_stock_data_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    df.to_csv(filename, index=False)
    print(f"Data saved to {filename}")

if __name__ == "__main__":
    main()

In [1]:
import praw
import pandas as pd
from datetime import datetime
import time

# Reddit API credentials (replace with your own)
reddit = praw.Reddit(
    client_id='9ezXtdh1pB2PhMu5oWrd1Q',
    client_secret='kZV7lztFgF9aUtNPzN0rS9lXReQ0xg',
    user_agent='u/Early_Impression5129'
)

subreddit_name = 'wallstreetbets'
keywords = ['buy', 'sell', 'invest', 'bullish', 'bearish']

# Function to extract potential stock tickers
def extract_tickers(text):
    tickers = []
    words = text.split()
    for word in words:
        if word.isupper() and len(word) <= 5:  # Simplistic check for stock tickers
            tickers.append(word)
    return ', '.join(tickers)

# Modified scrape_subreddit to include keyword filtering and ticker extraction
def scrape_subreddit(subreddit_name, post_limit=50000, comment_limit=10):
    subreddit = reddit.subreddit(subreddit_name)
    posts_data = []

    for post in subreddit.new(limit=post_limit):
        # Check if post contains any keywords
        if any(keyword in post.title.lower() or keyword in post.selftext.lower() for keyword in keywords):
            # Extract post data
            post_data = {
                'subreddit': subreddit_name,
                'post_id': post.id,
                'title': post.title,
                'body': post.selftext,
                'score': post.score,
                'num_comments': post.num_comments,
                'created_utc': datetime.fromtimestamp(post.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                'author': str(post.author),
                'url': post.url,
                'tickers': extract_tickers(post.title + ' ' + post.selftext)  # Extract tickers from title and body
            }
            
            # Extract comments
            post.comments.replace_more(limit=0)  # Flatten comment tree
            comments = []
            for comment in post.comments.list()[:comment_limit]:
                comments.append({
                    'comment_id': comment.id,
                    'comment_body': comment.body,
                    'comment_score': comment.score,
                    'comment_author': str(comment.author),
                    'comment_created_utc': datetime.fromtimestamp(comment.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                    'tickers': extract_tickers(comment.body)  # Extract tickers from comment
                })
            
            post_data['comments'] = comments
            posts_data.append(post_data)
        
        # Sleep to respect Reddit's rate limits
        time.sleep(2)

    return posts_data

def main():
    subreddits = ['stocks', 'investing', subreddit_name]
    all_data = []

    for subreddit in subreddits:
        print(f"Scraping r/{subreddit}...")
        subreddit_data = scrape_subreddit(subreddit)
        all_data.extend(subreddit_data)
        print(f"Finished scraping r/{subreddit}. Total posts scraped: {len(subreddit_data)}")

    # Convert to DataFrame, including extracted tickers
    df = pd.json_normalize(all_data, 'comments', ['subreddit', 'post_id', 'title', 'body', 'score', 'num_comments', 'created_utc', 'author', 'url', 'tickers'],
                           record_prefix='comment_')

    # Save to CSV
    filename = f"reddit_stock_data_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    df.to_csv(filename, index=False)
    print(f"Data saved to {filename}")

if __name__ == "__main__":
    main()


ModuleNotFoundError: No module named 'yfinance'

In [ ]:
df.head()

In [1]:
pip install yfinance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 23.8 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.6/84.6 kB 3.3 MB/s eta 0:00:00
  Created wheel for peewee: filename=peewee-3.17.6-cp310-cp310-linux_x86_64.whl size=293632 sha256=d1e838c8d364b235b3c7469fb56c90bb27f1ae03bf1018522c67da3c3c6dcbb9
  Stored in directory: /root/.cache/pip/wheels/4b/b9/b0/83d6e258e8f963f5ff111a2cd8c483ca59372a86e6a2535212
Successfully built peewee
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from textblob import TextBlob
import yfinance as yf

# Load the scraped Reddit data
df = pd.read_csv('/kaggle/working/reddit_stock_data_20240920_090900.csv')

def preprocess_data(df):
    # Convert dates to datetime using the correct format
    df['created_utc'] = pd.to_datetime(df['created_utc'], format='%Y-%m-%d %H:%M:%S')
    
    # Combine title and body for sentiment analysis
    df['full_text'] = df['title'] + ' ' + df['body'].fillna('')
    
    # Perform sentiment analysis using TextBlob
    df['sentiment'] = df['full_text'].apply(lambda x: TextBlob(x).sentiment.polarity)
    
    # Extract time-based features after converting datetime
    df['day_of_week'] = df['created_utc'].dt.dayofweek  # 0=Monday, 6=Sunday
    df['hour_of_day'] = df['created_utc'].dt.hour       # 0-23 hours
    
    return df

def get_stock_data(symbol, start_date, end_date):
    stock_data = yf.download(symbol, start=start_date, end=end_date)
    stock_data['returns'] = stock_data['Close'].pct_change()
    return stock_data

def create_features(reddit_df, stock_df):
    # Keep only the date part of 'created_utc' and extract time-based features
    reddit_df['date'] = reddit_df['created_utc'].dt.date
    reddit_df['day_of_week'] = reddit_df['created_utc'].dt.dayofweek  # 0=Monday, 6=Sunday
    reddit_df['hour_of_day'] = reddit_df['created_utc'].dt.hour       # 0-23 hours

    # Aggregate Reddit data by date, keeping time-based features
    daily_reddit = reddit_df.groupby('date').agg({
        'sentiment': 'mean',          # Average sentiment per day
        'score': 'sum',               # Sum of upvotes (popularity)
        'num_comments': 'sum',        # Sum of comments (engagement)
        'day_of_week': 'first',       # Take the first entry for day_of_week (since day is the same)
        'hour_of_day': 'first'        # Take the first entry for hour_of_day
    }).reset_index()

    # Prepare stock data, keeping only the date part
    stock_df['date'] = stock_df.index.date
    stock_df['returns'] = stock_df['Close'].pct_change()

    # Merge Reddit data with stock data on 'date'
    merged_df = pd.merge(daily_reddit, stock_df, left_on='date', right_on='date', how='inner')

    # Create target variable: 1 if next day's return is positive, 0 otherwise
    merged_df['target'] = (merged_df['returns'].shift(-1) > 0).astype(int)

    return merged_df.dropna()



def train_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.02, random_state=42)
    
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    print(classification_report(y_test, y_pred))
    
    return model

def main():
    # Load and preprocess Reddit data
    reddit_df = preprocess_data(df)
    
    # Get stock data (example: AAPL)
    start_date = reddit_df['created_utc'].min().strftime('%Y-%m-%d')
    end_date = reddit_df['created_utc'].max().strftime('%Y-%m-%d')
    stock_df = get_stock_data('AAPL', start_date, end_date)
    
    # Create features
    features_df = create_features(reddit_df, stock_df)
    
    # Prepare features and target
    X = features_df[['sentiment', 'score', 'num_comments', 'day_of_week', 'hour_of_day']]
    y = features_df['target']
    
    # Train and evaluate model
    model = train_model(X, y)
    
    print("Feature importances:")
    for feature, importance in zip(X.columns, model.feature_importances_):
        print(f"{feature}: {importance}")

if __name__ == "__main__":
    main()



In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from textblob import TextBlob
import yfinance as yf

# Load the scraped Reddit data
df = pd.read_csv('/kaggle/input/redditthous/reddit_stock_data_20240920_103445.csv')

def preprocess_data(df):
    # Convert dates to datetime using the correct format
    df['created_utc'] = pd.to_datetime(df['created_utc'], format='%Y-%m-%d %H:%M:%S')
    
    # Combine title and body for sentiment analysis
    df['full_text'] = df['title'] + ' ' + df['body'].fillna('')
    
    # Perform sentiment analysis using TextBlob
    df['sentiment'] = df['full_text'].apply(lambda x: TextBlob(x).sentiment.polarity)
    
    # Extract time-based features after converting datetime
    df['day_of_week'] = df['created_utc'].dt.dayofweek  # 0=Monday, 6=Sunday
    df['hour_of_day'] = df['created_utc'].dt.hour       # 0-23 hours
    
    return df

def get_stock_data(symbol, start_date, end_date):
    # Fetch historical stock data using yfinance
    stock_data = yf.download(symbol, start=start_date, end=end_date)
    stock_data['returns'] = stock_data['Close'].pct_change()  # Calculate daily returns
    return stock_data

def create_features(reddit_df, stock_df):
    # Aggregate Reddit data by date
    daily_reddit = reddit_df.groupby(reddit_df['created_utc'].dt.date).agg({
        'sentiment': 'mean',           # Average sentiment per day
        'score': 'sum',                # Sum of upvotes (popularity)
        'num_comments': 'sum',         # Sum of comments (engagement)
        'day_of_week': 'first',        # Get the first day_of_week entry for each day
        'hour_of_day': 'first'         # Get the first hour_of_day entry for each day
    }).reset_index()
    
    # Merge Reddit data with stock data
    stock_df['created_utc'] = stock_df.index.date
    merged_df = pd.merge(daily_reddit, stock_df, left_on='created_utc', right_on='created_utc', how='inner')
    
    # Create target variable: 1 if next day's return is positive, 0 otherwise
    merged_df['target'] = (merged_df['returns'].shift(-1) > 0).astype(int)
    
    return merged_df.dropna()

def train_model(X, y):
    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Train RandomForestClassifier model
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    # Predict on test set
    y_pred = model.predict(X_test)
    
    # Print classification report
    print(classification_report(y_test, y_pred))
    
    return model

def main():
    # Load and preprocess Reddit data
    reddit_df = preprocess_data(df)
    
    # Get stock data (example: AAPL)
    start_date = reddit_df['created_utc'].min().strftime('%Y-%m-%d')
    end_date = reddit_df['created_utc'].max().strftime('%Y-%m-%d')
    stock_df = get_stock_data('AAPL', start_date, end_date)
    
    # Create features by merging Reddit and stock data
    features_df = create_features(reddit_df, stock_df)
    
    # Prepare features and target variables for training
    X = features_df[['sentiment', 'score', 'num_comments', 'day_of_week', 'hour_of_day']]
    y = features_df['target']
    
    # Train the model and print results
    model = train_model(X, y)
    
    # Print feature importances from the trained model
    print("Feature importances:")
    for feature, importance in zip(X.columns, model.feature_importances_):
        print(f"{feature}: {importance}")

if __name__ == "__main__":
    main()


# Deep Learning

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from textblob import TextBlob
import yfinance as yf
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Moving Average working

In [7]:


# Load the scraped Reddit data
df = pd.read_csv('/kaggle/input/reddit10ksu/reddit_stock_data_20240920_192936.csv')

def preprocess_data(df):
    # Convert dates to datetime using the correct format
    df['created_utc'] = pd.to_datetime(df['created_utc'], format='%Y-%m-%d %H:%M:%S')
    
    # Combine title and body for sentiment analysis
    df['full_text'] = df['title'] + ' ' + df['body'].fillna('')
    
    # Perform sentiment analysis using TextBlob
    df['sentiment'] = df['full_text'].apply(lambda x: TextBlob(x).sentiment.polarity)
    
    # Extract time-based features after converting datetime
    df['day_of_week'] = df['created_utc'].dt.dayofweek  # 0=Monday, 6=Sunday
    df['hour_of_day'] = df['created_utc'].dt.hour       # 0-23 hours
    
    return df

def get_stock_data(symbol, start_date, end_date):
    # Fetch historical stock data using yfinance
    stock_data = yf.download(symbol, start=start_date, end=end_date)
    stock_data['returns'] = stock_data['Close'].pct_change()  # Calculate daily returns
    return stock_data

def create_features(reddit_df, stock_df):
    # Aggregate Reddit data by date and tickers
    daily_reddit = reddit_df.groupby([reddit_df['created_utc'].dt.date, 'tickers']).agg({
        'sentiment': 'mean',           # Average sentiment per day
        'score': 'sum',                # Sum of upvotes (popularity)
        'num_comments': 'sum',         # Sum of comments (engagement)
        'day_of_week': 'first',        # Get the first day_of_week entry for each day
        'hour_of_day': 'first'         # Get the first hour_of_day entry for each day
    }).reset_index()
    
    # Merge Reddit data with stock data
    stock_df['created_utc'] = stock_df.index.date
    merged_df = pd.merge(daily_reddit, stock_df, left_on='created_utc', right_on='created_utc', how='inner')
    
    # Calculate moving averages
    merged_df['MA_10'] = merged_df['Close'].rolling(window=10).mean()
    merged_df['MA_50'] = merged_df['Close'].rolling(window=50).mean()
    
    # Create result column based on moving average crossover strategy
    merged_df['result'] = (merged_df['MA_10'] > merged_df['MA_50']).astype(int)
    
    return merged_df.dropna()


def create_model(input_dim):
    model = Sequential()
    model.add(Dense(64, activation='relu', input_dim=input_dim))
    model.add(Dropout(0.3))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


def train_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Standardize the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Build and train the neural network model
    model = create_model(X_train_scaled.shape[1])
    model.fit(X_train_scaled, y_train, epochs=50, batch_size=16, validation_split=0.1, verbose=2)
    
    # Evaluate the model
    y_pred = (model.predict(X_test_scaled) > 0.5).astype(int)
    print(classification_report(y_test, y_pred))
    
    return model

def main():
    # Load and preprocess Reddit data
    reddit_df = preprocess_data(df)
    
    # Get stock data (example: AAPL)
    start_date = reddit_df['created_utc'].min().strftime('%Y-%m-%d')
    end_date = reddit_df['created_utc'].max().strftime('%Y-%m-%d')
    stock_df = get_stock_data('AAPL', start_date, end_date)
    
    # Create features by merging Reddit and stock data
    features_df = create_features(reddit_df, stock_df)
    
    # Prepare features and target variables for training
    X = features_df[['sentiment', 'score', 'num_comments', 'day_of_week', 'hour_of_day']]
    y = features_df['result']
    
    # Train the model and print results
    model = train_model(X, y)
    
    
if __name__ == "__main__":
    main()


[*********************100%***********************]  1 of 1 completed

Epoch 1/50



/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


45/45 - 2s - 36ms/step - accuracy: 0.5617 - loss: 0.6980 - val_accuracy: 0.5949 - val_loss: 0.6723
Epoch 2/50
45/45 - 0s - 3ms/step - accuracy: 0.6156 - loss: 0.6747 - val_accuracy: 0.5823 - val_loss: 0.6778
Epoch 3/50
45/45 - 0s - 3ms/step - accuracy: 0.6128 - loss: 0.6762 - val_accuracy: 0.5823 - val_loss: 0.6823
Epoch 4/50
45/45 - 0s - 3ms/step - accuracy: 0.5901 - loss: 0.6835 - val_accuracy: 0.5823 - val_loss: 0.6840
Epoch 5/50
45/45 - 0s - 3ms/step - accuracy: 0.6184 - loss: 0.6608 - val_accuracy: 0.5823 - val_loss: 0.6895
Epoch 6/50
45/45 - 0s - 3ms/step - accuracy: 0.6227 - loss: 0.6622 - val_accuracy: 0.5823 - val_loss: 0.6938
Epoch 7/50
45/45 - 0s - 3ms/step - accuracy: 0.6241 - loss: 0.6548 - val_accuracy: 0.5823 - val_loss: 0.6931
Epoch 8/50
45/45 - 0s - 3ms/step - accuracy: 0.6255 - loss: 0.6566 - val_accuracy: 0.5823 - val_loss: 0.6957
Epoch 9/50
45/45 - 0s - 3ms/step - accuracy: 0.6099 - loss: 0.6576 - val_accuracy: 0.5823 - val_loss: 0.6971
Epoch 10/50
45/45 - 0s - 3ms/

# Sentiment

In [8]:


# Load the scraped Reddit data
df = pd.read_csv('/kaggle/input/reddit10ksu/reddit_stock_data_20240920_192936.csv')

def preprocess_data(df):
    # Convert dates to datetime using the correct format
    df['created_utc'] = pd.to_datetime(df['created_utc'], format='%Y-%m-%d %H:%M:%S')
    
    # Combine title and body for sentiment analysis
    df['full_text'] = df['title'] + ' ' + df['body'].fillna('')
    
    # Perform sentiment analysis using TextBlob
    df['sentiment'] = df['full_text'].apply(lambda x: TextBlob(x).sentiment.polarity)
    
    # Extract time-based features after converting datetime
    df['day_of_week'] = df['created_utc'].dt.dayofweek  # 0=Monday, 6=Sunday
    df['hour_of_day'] = df['created_utc'].dt.hour       # 0-23 hours
    
    return df

def get_stock_data(symbol, start_date, end_date):
    # Fetch historical stock data using yfinance
    stock_data = yf.download(symbol, start=start_date, end=end_date)
    stock_data['returns'] = stock_data['Close'].pct_change()  # Calculate daily returns
    return stock_data

def create_features(reddit_df, stock_df):
    # Aggregate Reddit data by date and tickers
    daily_reddit = reddit_df.groupby([reddit_df['created_utc'].dt.date, 'tickers']).agg({
        'sentiment': 'mean',           # Average sentiment per day
        'score': 'sum',                # Sum of upvotes (popularity)
        'num_comments': 'sum',         # Sum of comments (engagement)
        'day_of_week': 'first',        # Get the first day_of_week entry for each day
        'hour_of_day': 'first'         # Get the first hour_of_day entry for each day
    }).reset_index()
    
    # Merge Reddit data with stock data
    stock_df['created_utc'] = stock_df.index.date
    merged_df = pd.merge(daily_reddit, stock_df, left_on='created_utc', right_on='created_utc', how='inner')
    
    # Create result column based on next day's average sentiment
    merged_df['result'] = (merged_df['sentiment'].shift(-1) > 0).astype(int)
    
    return merged_df.dropna()


def create_model(input_dim):
    model = Sequential()
    model.add(Dense(64, activation='relu', input_dim=input_dim))
    model.add(Dropout(0.3))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


def train_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Standardize the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Build and train the neural network model
    model = create_model(X_train_scaled.shape[1])
    model.fit(X_train_scaled, y_train, epochs=50, batch_size=16, validation_split=0.1, verbose=2)
    
    # Evaluate the model
    y_pred = (model.predict(X_test_scaled) > 0.5).astype(int)
    print(classification_report(y_test, y_pred))
    
    return model

def main():
    # Load and preprocess Reddit data
    reddit_df = preprocess_data(df)
    
    # Get stock data (example: AAPL)
    start_date = reddit_df['created_utc'].min().strftime('%Y-%m-%d')
    end_date = reddit_df['created_utc'].max().strftime('%Y-%m-%d')
    stock_df = get_stock_data('AAPL', start_date, end_date)
    
    # Create features by merging Reddit and stock data
    features_df = create_features(reddit_df, stock_df)
    
    # Prepare features and target variables for training
    X = features_df[['sentiment', 'score', 'num_comments', 'day_of_week', 'hour_of_day']]
    y = features_df['result']
    
    # Train the model and print results
    model = train_model(X, y)
    
    
if __name__ == "__main__":
    main()


[*********************100%***********************]  1 of 1 completed

Epoch 1/50



/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


47/47 - 2s - 33ms/step - accuracy: 0.6820 - loss: 0.6176 - val_accuracy: 0.8193 - val_loss: 0.4995
Epoch 2/50
47/47 - 0s - 3ms/step - accuracy: 0.8200 - loss: 0.5119 - val_accuracy: 0.8193 - val_loss: 0.4686
Epoch 3/50
47/47 - 0s - 3ms/step - accuracy: 0.8200 - loss: 0.4992 - val_accuracy: 0.8193 - val_loss: 0.4721
Epoch 4/50
47/47 - 0s - 3ms/step - accuracy: 0.8200 - loss: 0.4794 - val_accuracy: 0.8193 - val_loss: 0.4732
Epoch 5/50
47/47 - 0s - 3ms/step - accuracy: 0.8200 - loss: 0.4723 - val_accuracy: 0.8193 - val_loss: 0.4697
Epoch 6/50
47/47 - 0s - 3ms/step - accuracy: 0.8200 - loss: 0.4919 - val_accuracy: 0.8193 - val_loss: 0.4693
Epoch 7/50
47/47 - 0s - 3ms/step - accuracy: 0.8200 - loss: 0.4724 - val_accuracy: 0.8193 - val_loss: 0.4709
Epoch 8/50
47/47 - 0s - 3ms/step - accuracy: 0.8200 - loss: 0.4796 - val_accuracy: 0.8193 - val_loss: 0.4730
Epoch 9/50
47/47 - 0s - 3ms/step - accuracy: 0.8200 - loss: 0.4752 - val_accuracy: 0.8193 - val_loss: 0.4802
Epoch 10/50
47/47 - 0s - 3ms/

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# (Sentiment + Price Change)

In [9]:


# Load the scraped Reddit data
df = pd.read_csv('/kaggle/input/reddit10ksu/reddit_stock_data_20240920_192936.csv')

def preprocess_data(df):
    # Convert dates to datetime using the correct format
    df['created_utc'] = pd.to_datetime(df['created_utc'], format='%Y-%m-%d %H:%M:%S')
    
    # Combine title and body for sentiment analysis
    df['full_text'] = df['title'] + ' ' + df['body'].fillna('')
    
    # Perform sentiment analysis using TextBlob
    df['sentiment'] = df['full_text'].apply(lambda x: TextBlob(x).sentiment.polarity)
    
    # Extract time-based features after converting datetime
    df['day_of_week'] = df['created_utc'].dt.dayofweek  # 0=Monday, 6=Sunday
    df['hour_of_day'] = df['created_utc'].dt.hour       # 0-23 hours
    
    return df

def get_stock_data(symbol, start_date, end_date):
    # Fetch historical stock data using yfinance
    stock_data = yf.download(symbol, start=start_date, end=end_date)
    stock_data['returns'] = stock_data['Close'].pct_change()  # Calculate daily returns
    return stock_data

def create_features(reddit_df, stock_df):
    # Aggregate Reddit data by date and tickers
    daily_reddit = reddit_df.groupby([reddit_df['created_utc'].dt.date, 'tickers']).agg({
        'sentiment': 'mean',           # Average sentiment per day
        'score': 'sum',                # Sum of upvotes (popularity)
        'num_comments': 'sum',         # Sum of comments (engagement)
        'day_of_week': 'first',        # Get the first day_of_week entry for each day
        'hour_of_day': 'first'         # Get the first hour_of_day entry for each day
    }).reset_index()
    
    # Merge Reddit data with stock data
    stock_df['created_utc'] = stock_df.index.date
    merged_df = pd.merge(daily_reddit, stock_df, left_on='created_utc', right_on='created_utc', how='inner')
    
    # Calculate next day's return
    merged_df['next_day_return'] = merged_df['Close'].pct_change().shift(-1)
    
    # Create result column based on combined sentiment and next day return
    merged_df['result'] = ((merged_df['sentiment'] > 0) & (merged_df['next_day_return'] > 0)).astype(int)
    
    return merged_df.dropna()


def create_model(input_dim):
    model = Sequential()
    model.add(Dense(64, activation='relu', input_dim=input_dim))
    model.add(Dropout(0.3))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


def train_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Standardize the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Build and train the neural network model
    model = create_model(X_train_scaled.shape[1])
    model.fit(X_train_scaled, y_train, epochs=50, batch_size=16, validation_split=0.1, verbose=2)
    
    # Evaluate the model
    y_pred = (model.predict(X_test_scaled) > 0.5).astype(int)
    print(classification_report(y_test, y_pred))
    
    return model

def main():
    # Load and preprocess Reddit data
    reddit_df = preprocess_data(df)
    
    # Get stock data (example: AAPL)
    start_date = reddit_df['created_utc'].min().strftime('%Y-%m-%d')
    end_date = reddit_df['created_utc'].max().strftime('%Y-%m-%d')
    stock_df = get_stock_data('AAPL', start_date, end_date)
    
    # Create features by merging Reddit and stock data
    features_df = create_features(reddit_df, stock_df)
    
    # Prepare features and target variables for training
    X = features_df[['sentiment', 'score', 'num_comments', 'day_of_week', 'hour_of_day']]
    y = features_df['result']
    
    # Train the model and print results
    model = train_model(X, y)
    
    
if __name__ == "__main__":
    main()


[*********************100%***********************]  1 of 1 completed

Epoch 1/50



/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


47/47 - 2s - 37ms/step - accuracy: 0.9295 - loss: 0.4162 - val_accuracy: 0.9639 - val_loss: 0.2465
Epoch 2/50
47/47 - 0s - 3ms/step - accuracy: 0.9797 - loss: 0.1598 - val_accuracy: 0.9639 - val_loss: 0.1745
Epoch 3/50
47/47 - 0s - 3ms/step - accuracy: 0.9797 - loss: 0.1161 - val_accuracy: 0.9639 - val_loss: 0.1782
Epoch 4/50
47/47 - 0s - 3ms/step - accuracy: 0.9797 - loss: 0.1151 - val_accuracy: 0.9639 - val_loss: 0.1854
Epoch 5/50
47/47 - 0s - 3ms/step - accuracy: 0.9797 - loss: 0.1089 - val_accuracy: 0.9639 - val_loss: 0.1825
Epoch 6/50
47/47 - 0s - 3ms/step - accuracy: 0.9797 - loss: 0.1142 - val_accuracy: 0.9639 - val_loss: 0.1863
Epoch 7/50
47/47 - 0s - 3ms/step - accuracy: 0.9797 - loss: 0.1087 - val_accuracy: 0.9639 - val_loss: 0.1866
Epoch 8/50
47/47 - 0s - 3ms/step - accuracy: 0.9797 - loss: 0.1097 - val_accuracy: 0.9639 - val_loss: 0.1872
Epoch 9/50
47/47 - 0s - 3ms/step - accuracy: 0.9797 - loss: 0.1083 - val_accuracy: 0.9639 - val_loss: 0.1862
Epoch 10/50
47/47 - 0s - 3ms/

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# claude

In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from textblob import TextBlob
import yfinance as yf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

# Load the scraped Reddit data
df = pd.read_csv('/kaggle/input/reddit10ksu/reddit_stock_data_20240920_192936.csv')

def preprocess_data(df):
    # Convert dates to datetime using the correct format
    df['created_utc'] = pd.to_datetime(df['created_utc'], errors='coerce')
    
    # Combine title and body for sentiment analysis
    df['full_text'] = df['title'] + ' ' + df['body'].fillna('')
    
    # Perform sentiment analysis using TextBlob with error handling
    df['sentiment'] = df['full_text'].apply(lambda x: TextBlob(x).sentiment.polarity if x else 0)
    
    # Extract time-based features after converting datetime
    df['day_of_week'] = df['created_utc'].dt.dayofweek  # 0=Monday, 6=Sunday
    df['hour_of_day'] = df['created_utc'].dt.hour       # 0-23 hours
    
    return df

def get_stock_data(tickers, start_date, end_date):
    stock_data = {}
    for ticker in tickers:
        try:
            data = yf.download(ticker, start=start_date, end=end_date)
            data['returns'] = data['Close'].pct_change()
            stock_data[ticker] = data
        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")
    return stock_data

def categorize_returns(returns):
    if returns <= -0.02:
        return 0  # Significant decrease
    elif -0.02 < returns < -0.005:
        return 1  # Moderate decrease
    elif -0.005 <= returns <= 0.005:
        return 2  # Minimal change
    elif 0.005 < returns < 0.02:
        return 3  # Moderate increase
    else:
        return 4  # Significant increase

def create_features(reddit_df, stock_data):
    all_features = []
    
    for ticker in stock_data.keys():
        # Filter Reddit data for this ticker
        ticker_df = reddit_df[reddit_df['tickers'].str.contains(ticker, na=False)]
        
        # Aggregate Reddit data by date
        daily_reddit = ticker_df.groupby(ticker_df['created_utc'].dt.date).agg({
            'sentiment': 'mean',
            'score': 'sum',
            'num_comments': 'sum',
            'day_of_week': 'first',
            'hour_of_day': 'first'
        }).reset_index()
        
        # Merge Reddit data with stock data
        stock_df = stock_data[ticker].reset_index()
        stock_df['created_utc'] = pd.to_datetime(stock_df['Date']).dt.date
        merged_df = pd.merge(daily_reddit, stock_df, on='created_utc', how='inner')
        
        # Create target variable: categorized next day's return
        merged_df['target'] = merged_df['returns'].shift(-1).apply(categorize_returns)
        merged_df['ticker'] = ticker
        
        all_features.append(merged_df)
    
    return pd.concat(all_features, ignore_index=True).dropna()

def create_model(input_dim, num_classes):
    model = Sequential()
    model.add(Dense(64, activation='relu', input_dim=input_dim))
    model.add(Dropout(0.3))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(num_classes, activation='softmax'))
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def train_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Standardize the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Convert to categorical
    y_train_cat = to_categorical(y_train)
    y_test_cat = to_categorical(y_test)
    
    # Build and train the neural network model
    model = create_model(X_train_scaled.shape[1], y_train_cat.shape[1])
    
    # Add EarlyStopping to prevent overfitting
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    
    model.fit(X_train_scaled, y_train_cat, epochs=50, batch_size=32, 
              validation_split=0.1, callbacks=[early_stopping], verbose=2)
    
    # Evaluate the model
    y_pred = model.predict(X_test_scaled)
    y_pred_classes = np.argmax(y_pred, axis=1)
    print(classification_report(y_test, y_pred_classes))
    
    return model, scaler

def main():
    # Load and preprocess Reddit data
    reddit_df = preprocess_data(df)
    
    # Get unique tickers from the dataset
    tickers = reddit_df['tickers'].dropna().str.split(',').explode().unique()
    tickers = [ticker.strip() for ticker in tickers if ticker.strip()]
    
    # Get stock data for all tickers
    start_date = reddit_df['created_utc'].min().strftime('%Y-%m-%d')
    end_date = reddit_df['created_utc'].max().strftime('%Y-%m-%d')
    stock_data = get_stock_data(tickers, start_date, end_date)
    
    # Create features by merging Reddit and stock data
    features_df = create_features(reddit_df, stock_data)
    
    # Prepare features and target variables for training
    X = features_df[['sentiment', 'score', 'num_comments', 'day_of_week', 'hour_of_day']]
    y = features_df['target']
    
    # Train the model and print results
    model, scaler = train_model(X, y)
    
    print("Model training completed.")
    
    # Example prediction for the latest data point of each ticker
    for ticker in stock_data.keys():
        latest_data = features_df[features_df['ticker'] == ticker].iloc[-1]
        latest_features = latest_data[['sentiment', 'score', 'num_comments', 'day_of_week', 'hour_of_day']].values.reshape(1, -1)
        scaled_features = scaler.transform(latest_features)
        prediction = model.predict(scaled_features)[0]
        predicted_class = np.argmax(prediction)
        class_probabilities = prediction[predicted_class]
        
        movement_labels = ['Significant decrease', 'Moderate decrease', 'Minimal change', 'Moderate increase', 'Significant increase']
        print(f"Prediction for {ticker}: {movement_labels[predicted_class]} (Probability: {class_probabilities:.2f})")

if __name__ == "__main__":
    main()


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

KeyboardInterrupt: 